## 유니코드 정규화(Unicode Normalization) 정리

유니코드 정규화의 **네 가지 주요 방식**(NFC, NFD, NFKC, NFKD)을 설명하는 마크다운 문서입니다. 각 방식의 개념, 예시 코드, 사용 시점 등을 한국어로 정리했습니다.

---

### 개요

유니코드 정규화는 "겉으로는 같은" 문자가 내부적으로 다른 코드 포인트 조합으로 표현되는 문제를 해결하기 위해 **문자 표현을 통일**하는 과정입니다. 대표적으로 다음 네 가지 방식이 있습니다.

* **NFC (Normalization Form C)**: 가능한 경우 조합(composed)하여 하나의 선형 코드 포인트로 만듭니다.
* **NFD (Normalization Form D)**: 가능한 경우 분해(decomposed)하여 기본 문자와 결합 문자로 나눕니다.
* **NFKC (Compatibility Composition)**: 호환성(겉모양은 유사하지만 다른 문자인 경우) 문자를 호환 가능한 기본 형태로 매핑한 뒤 조합합니다.
* **NFKD (Compatibility Decomposition)**: 호환성 매핑 후 분해합니다.

---

### 예시 (문자: `é`)

* 단일 문자: `U+00E9` (é)
* 분해된 형태: `U+0065` (e) + `U+0301` (combining acute accent)

|           입력 형태 |  NFC 결과  |      NFD 결과     |  NFKC 결과 |     NFKD 결과     |
| --------------: | :------: | :-------------: | :------: | :-------------: |
|        `U+00E9` | `U+00E9` | `U+0065 U+0301` | `U+00E9` | `U+0065 U+0301` |
| `U+0065 U+0301` | `U+00E9` | `U+0065 U+0301` | `U+00E9` | `U+0065 U+0301` |

---

### 파이썬 예제

```python
import unicodedata

s1 = 'é'                 # U+00E9
s2 = 'e\u0301'          # e + combining acute accent

for form in ('NFC', 'NFD', 'NFKC', 'NFKD'):
    ns1 = unicodedata.normalize(form, s1)
    ns2 = unicodedata.normalize(form, s2)
    print(form, ns1 == ns2, [hex(ord(ch)) for ch in ns1])
```

출력 예시(요약):

```
NFC  True  ['0xe9']
NFD  True  ['0x65', '0x301']
NFKC True  ['0xe9']
NFKD True  ['0x65', '0x301']
```

---

### 언제 어떤 정규화를 사용할까?

* **일반 텍스트 저장/교환**: `NFC` 권장 (많은 플랫폼이 NFC를 기본으로 사용).
* **텍스트 비교/검색/인덱싱**: 입력과 DB를 동일한 정규화 방식으로 맞춘 뒤 비교. (예: 저장 전 `NFC`로 통일)
* **호환 문자 통일(숫자 원번호, 분수, 위 첨자 등)**: 시각적 호환성을 제거하려면 `NFKC` 또는 `NFKD` 사용 (주의: 의미가 변할 수 있음).
* **OS 파일명 호환**: macOS는 NFD 계열을 사용하는 경우가 많으므로 교차 플랫폼에서 파일명 비교 시 정규화 필요.

---

### 주의사항

* `NFKC`/`NFKD`는 "호환성 매핑"을 수행해 의미나 표현이 바뀔 수 있으므로 민감한 원문(예: 법적 문서, 암호)에는 사용하지 않는 것이 안전합니다.
* 정규화는 시각적 동등성을 보장하지만, 언어학적 동등성(동일한 의미)을 보장하지는 않습니다.

---

### 요약

* **NFC**: 가능한 조합 -> 일반 저장/교환에 적합
* **NFD**: 가능한 분해 -> 문자 분석이나 조합문자 처리에 적합
* **NFKC/NFKD**: 호환성 매핑 포함 -> 검색/정규화된 비교에 유리하지만 의미 변형 주의

필요하면 이 마크다운을 README용으로 다듬어주거나, 추가로 **macOS/Windows 파일명 예시**, **데이터베이스 설정 예시 (MySQL/Postgres)**, **정규화 자동화 스크립트** 등을 만들어줄게.

---

## 정규화를 사용해야 하는 이유

유니코드 정규화가 필요한 이유는 **겉으로는 동일하게 보이지만 내부 표현이 다른 문자들을 일관되게 처리하기 위해서**입니다.

### 1. 동일한 문자가 내부적으로 다른 코드 포인트로 표현될 수 있음

* 예: `é`

  * 조합형: `e (U+0065)` + `´ (U+0301)`
  * 단일형: `U+00E9`
* 화면에서는 같아도 문자열 비교 시 다르게 판단됨

```
"é" == "é"  # False
```

### 2. 검색, 정렬, 비교 기능이 정확하게 동작하도록 함

* 검색 엔진, DB 인덱스가 서로 다른 정규화 상태에서는 일관되지 않은 결과를 반환할 수 있음
* 정규화를 통일하면 검색/비교가 정확해짐

### 3. 운영체제 간 파일명 호환성 문제 해결

* macOS: NFD 기반 저장
* Windows: NFC 기반 저장
* 정규화가 서로 다르면 "같은 이름인데 다른 파일로 인식"되는 문제가 발생할 수 있음

### 4. 데이터베이스 저장 시 충돌 방지

* 서로 다른 정규화 방식으로 저장된 데이터는 중복 체크가 실패하거나, 인덱스가 올바르게 작동하지 않을 수 있음

### 5. 사용자 입력을 표준화하여 안정적인 처리 가능

* 키보드, 프로그램, 웹 입력 등 다양한 경로에서 유입된 텍스트가 서로 다른 유니코드 구조일 수 있음
* 정규화로 데이터의 일관성을 확보할 수 있음

### 요약

* **정확한 비교**
* **데이터 일관성 유지**
* **OS/DB 간 호환성 확보**
* **검색·정렬 성능 및 정확도 향상**

정규화는 현대의 텍스트 처리에서 필수적인 과정입니다.
